In [1]:
import os
import pandas as pd
%run ./preprocess.ipynb

ds_root = '/home/gaurav.gupta/projects/PoCs/brandMention/brand_datasets/'
path = os.path.join(ds_root,'ds_complaints', 'panasonic_v1_g.csv')
path = os.path.join(ds_root, 'ds_complaints', 'panasonic_random_sample_predicted.csv')

m_df = read_file(path)
r_df = read_file(path)
df = pd.concat([r_df, m_df])
pdf = process_data(df)
pdf.head(2)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gaurav.gupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


------------------  -----------  ------------
Step                Total words  Unique words
Start               55158        9445
Lower               55158        8619
Lemmatize           55158        8283
Unicode Fix         55194        8281
Replace emoji       55730        8375
Stop words          45710        8323
Email Replace       45710        8319
UserName replace    45818        7771
HashTags Replace    58132        5426
URL Replace         58132        4478
MARKUP Replace      58132        4478
Remove punctuation  58132        3729
------------------  -----------  ------------


,text,Complaint
0,film HASHTAG URL special HASHTAG foil adapte...,0
1,HANDLE centre awesome collection HANDLE plush...,0


In [4]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

pdf['Complaint'] = pdf['Complaint'].astype(int)
pdf['text'] = pdf['text'].astype(str)
pdf.columns

# x_train, x_test, y_train, y_test = train_test_split(pdf['text'], pdf['Complaint'], random_state=42, test_size=0.30)  # 1
epochs=5
batch_size=64
MAX_NB_WORDS = 500
MAX_SEQUENCE_LENGTH = 10000


model = keras.Sequential()
model.add(layers.Embedding(input_dim=MAX_SEQUENCE_LENGTH, output_dim=64))
model.add(layers.LSTM(128))
model.add(layers.Dense(2))
model.summary()

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(pdf['text'].values)
word_index = tokenizer.word_index

X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
Y = pdf['Complaint']

# print(X.shape)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)
# print(x_train.shape)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# history = model.fit(x_train, 
#                     y_train, 
#                     epochs=epochs, 
#                     batch_size=batch_size, 
#                     validation_split=0.1, 
#                     callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]
#                     )


2021-12-28 22:40:20.736390: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 739,074
Trainable params: 739,074
Non-trainable params: 0
_________________________________________________________________


2021-12-28 22:40:21.223125: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-28 22:40:21.241767: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1382400000 Hz


Epoch 1/5
 4/36 [==>...........................] - ETA: 7:35 - loss: 0.0370 - accuracy: 0.5361

KeyboardInterrupt: 